
# Classificação de Direção de Ações Brasileiras com Machine Learning

Este notebook apresenta um pipeline completo de classificação para prever a direção da variação percentual diária do preço de fechamento de ações brasileiras. Utilizaremos dados históricos de cinco ativos: BBAS3, EMBR3, TOTS3, PETR4 e KLBN3. O objetivo é construir e comparar modelos capazes de prever se a variação do próximo dia será positiva (alta) ou negativa (baixa).



## 1. Carregamento e Limpeza dos Dados


In [ ]:

import pandas as pd

tickers_files = {
    'BBAS3': 'BBAS3_Dados_Históricos.csv',
    'EMBR3': 'EMBR3_Dados_Históricos.csv',
    'TOTS3': 'TOTS3_Dados_Históricos.csv',
    'PETR4': 'PETR4_Dados_Históricos.csv',
    'KLBN3': 'KLBN3_Dados_Históricos.csv'
}

def load_and_clean(file, ticker):
    df = pd.read_csv(file, sep='|')
    df.columns = [col.strip() for col in df.columns]
    data_col = [col for col in df.columns if 'data' in col.lower()][0]
    df['Data'] = pd.to_datetime(df[data_col], format='%d.%m.%Y')
    for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
        col_found = [c for c in df.columns if col.lower() in c.lower()]
        if col_found:
            df[col] = df[col_found[0]].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    vol_col = [c for c in df.columns if 'vol' in c.lower()][0]
    df['Vol.'] = df[vol_col].astype(str).str.replace('.', '', regex=False).str.replace('K', 'e3').str.replace('M', 'e6').apply(pd.eval)
    var_col = [c for c in df.columns if 'var' in c.lower()][0]
    df['Var%'] = df[var_col].astype(str).str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    df['Ticker'] = ticker
    return df[['Data','Último','Abertura','Máxima','Mínima','Vol.','Var%','Ticker']]

dfs = []
for ticker, file in tickers_files.items():
    dfs.append(load_and_clean(file, ticker))
data = pd.concat(dfs, ignore_index=True)
data = data.sort_values(['Ticker', 'Data'])
data.head()



## 2. Análise Exploratória dos Dados (EDA)


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

display(data.describe())

plt.figure(figsize=(12,6))
sns.histplot(data['Var%'], bins=50, kde=True)
plt.title('Distribuição da Variação Percentual Diária')
plt.show()

plt.figure(figsize=(14,7))
for ticker in tickers_files.keys():
    subset = data[data['Ticker'] == ticker]
    plt.plot(subset['Data'], subset['Último'], label=ticker)
plt.legend()
plt.title('Preço de Fechamento ao Longo do Tempo')
plt.show()



## 3. Pré-processamento e Feature Engineering


In [ ]:

def add_lags(df, n_lags=3):
    for lag in range(1, n_lags+1):
        df[f'Var%_lag{lag}'] = df['Var%'].shift(lag)
        df[f'Último_lag{lag}'] = df['Último'].shift(lag)
    return df

data = data.groupby('Ticker').apply(add_lags).reset_index(drop=True)
data = data.dropna()

# Criar coluna de target binária: 1 para alta, 0 para baixa
# (considerando 0 como neutro, pode ser removido se preferir)
data['Target'] = (data['Var%'] > 0).astype(int)

features = [col for col in data.columns if 'lag' in col]
target = 'Target'
data[features + [target]].head()



## 4. Divisão de Dados em Treino e Teste


In [ ]:

def split_train_test(df, frac=0.8):
    idx = int(len(df)*frac)
    return df.iloc[:idx], df.iloc[idx:]

train = []
test = []
for ticker in tickers_files.keys():
    t, s = split_train_test(data[data['Ticker'] == ticker])
    train.append(t)
    test.append(s)
train = pd.concat(train)
test = pd.concat(test)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

print('Exemplo de X_train:')
display(X_train.head())
print('Distribuição do target no treino:')
print(y_train.value_counts(normalize=True))



## 5. Treinamento e Avaliação de Modelos

Vamos comparar Random Forest, Regressão Logística e SVM.


In [ ]:

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Random Forest:')
print(classification_report(y_test, y_pred_rf))
print('Matriz de confusão:')
print(confusion_matrix(y_test, y_pred_rf))


In [ ]:

# Regressão Logística
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('Logistic Regression:')
print(classification_report(y_test, y_pred_lr))
print('Matriz de confusão:')
print(confusion_matrix(y_test, y_pred_lr))


In [ ]:

# SVM
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print('SVM:')
print(classification_report(y_test, y_pred_svc))
print('Matriz de confusão:')
print(confusion_matrix(y_test, y_pred_svc))



## 6. Comparação dos Modelos


In [ ]:

results = pd.DataFrame({
    'Random Forest': [accuracy_score(y_test, y_pred_rf)],
    'Logistic Regression': [accuracy_score(y_test, y_pred_lr)],
    'SVM': [accuracy_score(y_test, y_pred_svc)]
}, index=['Acurácia'])
display(results)



## 7. Importância das Features (Random Forest)


In [ ]:

importances = rf.feature_importances_
feat_imp = pd.Series(importances, index=features).sort_values(ascending=False)
feat_imp.plot(kind='bar', figsize=(12,5), title='Importância das Features')
plt.show()



## 8. Salvando o Modelo Treinado


In [ ]:

import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(rf, f)
print('Modelo salvo como model.pkl')



## 9. Conclusão e Próximos Passos

Neste notebook, realizamos a classificação da direção da variação percentual diária de ações brasileiras, incluindo:

- Carregamento e limpeza robusta dos dados históricos.
- Análise exploratória e visualização dos dados.
- Feature engineering com lags e criação de target binário.
- Treinamento e avaliação de três modelos: Random Forest, Regressão Logística e SVM.
- Comparação quantitativa dos resultados.
- Análise de importância das features.
- Salvamento do modelo treinado para uso futuro.

**Próximos passos:**
- Testar outros algoritmos (ex: XGBoost, LightGBM).
- Realizar tuning de hiperparâmetros.
- Incluir mais features (ex: indicadores técnicos).
- Avaliar o modelo em dados mais recentes ou em produção.

*Este notebook serve como base para experimentação e pode ser expandido conforme a necessidade do projeto.*
